In [9]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days, date_add_times
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths = glob('../feature/eda_base/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count('time_zone')
               or path.count('hour')
               or path.count('Product')
              ]
test_paths = [path for path in test_paths 
               if path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count('time_zone')
               or path.count('hour')
               or path.count('Product')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
if COLUMN_ID in data.columns:
    data.set_index(COLUMN_ID, inplace=True)
    
del df_train, df_test
gc.collect()

base_train = read_pkl_gzip('../input/base_train.gz').set_index(COLUMN_ID)
base_test = read_pkl_gzip('../input/base_test.gz').set_index(COLUMN_ID)
length_train = base_train.shape[0]
base = pd.concat([base_train, base_test], axis=0)

In [19]:
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')

data['datetime'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
data['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
data['date'] = data['datetime'].map(lambda x: x.date())
data['hour'] = data['datetime'].map(lambda x: x.hour)
data['date-hour'] = data[['date', 'hour']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)

In [38]:
max_datetime
# df[df['date-hour']==end_date]
str(type(max_datetime))

"<class 'pandas._libs.tslibs.nattype.NaTType'>"

In [46]:
#========================================================================
# FE Aggregation User ID & TimeSeries Date
#========================================================================

# n_hour = 3
# for i in range(1):

def parallel_agg(df, base_key, n_hour, feature, name_feature):
    list_term_df = []
    for end_date in tqdm(list_end_datetime):
        
        max_datetime = df[df['date-hour']==end_date]['datetime'].max()
        if str(type(max_datetime)).count('NaT'):
            continue
        
        start_datetime = date_add_times(max_datetime, n_hour*-1)
        tmp_user = df[df['date-hour']==end_date][[base_key]].drop_duplicates()
        
        # 未来を計算させる場合はstart, endの不等号を逆にする
        if n_hour>0:
            tmp = df[(start_datetime <= df.datetime) & (df.datetime <= max_datetime)]
        else:
            tmp = df[(max_datetime <= df.datetime) & (df.datetime <= start_datetime)]

        result = tmp.groupby(base_key, as_index=False)[feature].agg({
            f'{base_key}-{name_feature}_hour{n_hour}_{feature}_count': 'count',
        })

        tmp_user = tmp_user.merge(result, on=base_key, how='inner')
        tmp_user['date-hour'] = end_date
        list_term_df.append(tmp_user)
    df_agg = pd.concat(list_term_df, axis=0)
    df_agg.set_index([base_key, 'date-hour'], inplace=True)
    return df_agg


# User別に期間を切って集計できる様にする
prefix = '602'
dir_save = 'valid'
feature = 'date-hour'
base_key = 'ProductCD'

list_base_key = [col for col in data.columns if col.count('user_id')]
list_end_datetime = sorted(data['date-hour'].unique())
list_base_date = list_end_datetime

tmp_n_hour = np.array([1, 3, 6, 12, 18, 24])
# numpy.intだとエラーでる、、
list_n_hour = np.append(tmp_n_hour, np.array(tmp_n_hour)*-1).tolist()
list_product = data['ProductCD'].unique()

for product in list_product.tolist() + ['all']:
    base_train = data[[base_key, 'date-hour']].iloc[:length_train]
    base_test  = data[[base_key, 'date-hour']].iloc[length_train:]
    name_feature = product
    
    if product != 'all':
        df = data[data['ProductCD']==product]
    else:
        df = data.copy()

    list_p = Parallel(len(list_n_hour))(
        [delayed(parallel_agg)(
            df[[base_key, 'date-hour', 'datetime']], base_key, n_hour, feature, name_feature
        ) for n_hour in list_n_hour])

    df_agg = pd.concat(list_p, axis=1)
    df_agg.reset_index(inplace=True)

    base_train_agg = base_train.merge(df_agg, how='left', on=[base_key, 'date-hour'])
    base_test_agg = base_test.merge(df_agg, how='left', on=[base_key, 'date-hour'])

    cols_save = [col for col in base_train_agg.columns if col.count(f'{name_feature}_hour')]

    save_feature(base_train_agg[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
    save_feature(base_test_agg[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | ProductCD-W_hour1_date-hour_count
(590540,) | ProductCD-W_hour3_date-hour_count
(590540,) | ProductCD-W_hour6_date-hour_count
(590540,) | ProductCD-W_hour12_date-hour_count
(590540,) | ProductCD-W_hour18_date-hour_count
(590540,) | ProductCD-W_hour24_date-hour_count
(590540,) | ProductCD-W_hour-1_date-hour_count
(590540,) | ProductCD-W_hour-3_date-hour_count
(590540,) | ProductCD-W_hour-6_date-hour_count
(590540,) | ProductCD-W_hour-12_date-hour_count
(590540,) | ProductCD-W_hour-18_date-hour_count
(590540,) | ProductCD-W_hour-24_date-hour_count
(506691,) | ProductCD-W_hour1_date-hour_count
(506691,) | ProductCD-W_hour3_date-hour_count
(506691,) | ProductCD-W_hour6_date-hour_count
(506691,) | ProductCD-W_hour12_date-hour_count
(506691,) | ProductCD-W_hour18_date-hour_count
(506691,) | ProductCD-W_hour24_date-hour_count
(506691,) | ProductCD-W_hour-1_date-hour_count
(506691,) | ProductCD-W_hour-3_date-hour_count
(506691,) | ProductCD-W_hour-6_date-hour_count
(506691,) | Prod

In [48]:
base_train_agg

,ProductCD,date-hour,ProductCD-all_hour1_date-hour_count,ProductCD-all_hour3_date-hour_count,ProductCD-all_hour6_date-hour_count,ProductCD-all_hour12_date-hour_count,ProductCD-all_hour18_date-hour_count,ProductCD-all_hour24_date-hour_count,ProductCD-all_hour-1_date-hour_count,ProductCD-all_hour-3_date-hour_count,ProductCD-all_hour-6_date-hour_count,ProductCD-all_hour-12_date-hour_count,ProductCD-all_hour-18_date-hour_count,ProductCD-all_hour-24_date-hour_count
0,W,2017-12-01-20,166,166,166,166,166,166,146.0,345.0,511.0,888.0,2495.0,4159.0
1,W,2017-12-01-20,166,166,166,166,166,166,146.0,345.0,511.0,888.0,2495.0,4159.0
2,W,2017-12-01-20,166,166,166,166,166,166,146.0,345.0,511.0,888.0,2495.0,4159.0
3,W,2017-12-01-20,166,166,166,166,166,166,146.0,345.0,511.0,888.0,2495.0,4159.0
4,H,2017-12-01-20,25,25,25,25,25,25,19.0,57.0,73.0,92.0,224.0,359.0
5,W,2017-12-01-20,166,166,166,166,166,166,146.0,345.0,511.0,888.0,2495.0,4159.0
6,W,2017-12-01-20,166,166,166,166,166,166,146.0,345.0,511.0,888.0,2495.0,4159.0
7,W,2017-12-01-20,166,166,166,166,166,166,146.0,345.0,511.0,888.0,2495.0,4159.0
8,H,2017-12-01-20,25,25,25,25,25,25,19.0,57.0,73.0,92.0,224.0,359.0
9,W,2017-12-01-20,166,166,166,166,166,166,146.0,345.0,511.0,888.0,2495.0,4159.0
